In [ ]:
# ==============================================================================
# CÉLULA 1: SETUP E INICIALIZAÇÃO DA VERSÃO FINAL
# ==============================================================================
import pandas as pd
from pathlib import Path
import sys
import time

print("--- 1. INICIANDO SETUP DE TESTES ---")

# --- AJUSTE DE PATH PARA IMPORTAÇÃO ---
# Como o notebook está em 'notebooks/', a raiz do projeto está um nível acima ('..')
# E o nosso código-fonte está em 'src/'
project_root = Path('.').resolve().parent
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    print(f"Adicionando diretório '{src_path}' ao path do sistema.")
    sys.path.append(str(src_path))

# --- IMPORTAÇÃO CORRIGIDA ---
# Agora importamos do pacote 'b3_analyzer' e do módulo 'analyzer'
from b3_analyzer.analyzer import B3Data

# Configurações de exibição do Pandas
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 150)

# Inicializa o Analisador
# Os caminhos agora são construídos a partir da raiz do projeto
processed_data_dir = project_root / 'data' / 'processed'
analyzer = B3Data(data_path=str(processed_data_dir))

print("\n--- SETUP CONCLUÍDO. ANALISADOR FINAL PRONTO PARA TESTE DE ESTRESSE. ---")
print("-" * 80)

In [ ]:
# ==============================================================================
# CÉLULA 2: TESTES DA CAMADA 1 - ACESSO DIRETO E RÁPIDO
# ==============================================================================
# Descrição: Validamos os casos de uso do "Analista Quantitativo", que precisa
# de acesso rápido a dados de tickers específicos.
# ------------------------------------------------------------------------------

print("--- TESTE 1.1: BUSCA SIMPLES POR TICKER PARA UMA CARTEIRA ---")
carteira = ['ITSA4', 'TAEE11', 'BBSE3']
df_carteira = analyzer.get_quotes(
    tickers=carteira,
    start_date='2023-01-01',
    end_date='2023-01-05',
    columns=['PREULT', 'VOLTOT']
)
print(f"Resultado para a carteira {carteira}:")
display(df_carteira)
assert not df_carteira.empty, "Teste 1.1 Falhou: Busca simples não retornou dados."
print("[OK] Teste 1.1 Concluído com sucesso.\n" + "-"*80)


print("\n--- TESTE 1.2: FILTRO GRANULAR POR ESPECIFICAÇÃO (PADRÃO) ---")
# Objetivo: Pegar apenas as ações Preferenciais (PN) da Gerdau, ignorando as outras.
df_ggbr_pn = analyzer.get_quotes(
    tickers=['GGBR3', 'GGBR4'],
    start_date='2023-03-01',
    end_date='2023-03-05',
    especificacao='PN' # Deve filtrar e retornar apenas dados da GGBR4
)
print("Resultado para GGBR com filtro 'PN':")
display(df_ggbr_pn)
assert df_ggbr_pn['CODNEG'].unique() == ['GGBR4'], "Teste 1.2 Falhou: Filtro de especificação não funcionou."
print("[OK] Teste 1.2 Concluído com sucesso.\n" + "-"*80)

In [ ]:
# ==============================================================================
# CÉLULA 3: TESTES DA CAMADA 2 - EXPLORAÇÃO INTELIGENTE
# ==============================================================================
# Descrição: Validamos os novos recursos de busca por entidade e classe de ativo.
# ------------------------------------------------------------------------------

print("\n--- TESTE 2.1: BUSCA UNIVERSAL COM find_assets() ---")
# Objetivo: Encontrar todos os ativos relacionados à "CEMIG".
df_info_cemig = analyzer.find_assets(query='CEMIG')
print("Ativos encontrados para a query 'CEMIG':")
display(df_info_cemig)
assert not df_info_cemig.empty, "Teste 2.1 Falhou: find_assets não encontrou a Cemig."
print("[OK] Teste 2.1 Concluído com sucesso.\n" + "-"*80)


print("\n--- TESTE 2.2: BUSCA POR ENTIDADE EM get_quotes() ---")
# Objetivo: Obter todas as cotações da "Ambev" sem saber o ticker.
df_ambev = analyzer.get_quotes(
    entity='AMBEV',
    start_date='2023-04-03',
    end_date='2023-04-06'
)
print("Resultado da busca pela entidade 'AMBEV':")
display(df_ambev)
assert 'ABEV3' in df_ambev['CODNEG'].unique(), "Teste 2.2 Falhou: Busca por entidade não encontrou o ticker ABEV3."
print("[OK] Teste 2.2 Concluído com sucesso.\n" + "-"*80)

In [ ]:
# ==============================================================================
# CÉLULA 4: TESTES DE CASOS EXTREMOS E COMBINAÇÕES (VERSÃO ROBUSTA)
# ==============================================================================
# Descrição: Forçamos o módulo a lidar com buscas complexas, incluindo uma
# busca mais ampla por BDRs para garantir que a lógica de filtragem funcione.
# ------------------------------------------------------------------------------

print("\n--- TESTE 3.1 (EXTREMO): BUSCA POR CLASSE DE ATIVO EM UM UNIVERSO AMPLO ---")
# Objetivo: Encontrar TODOS os BDRs negociados em um MÊS INTEIRO para validar a lógica.
start_time = time.time()
df_bdrs = analyzer.get_quotes(
    asset_class='bdr',
    start_date='2023-05-01', # <-- ALTERADO para um range maior
    end_date='2023-05-31',   # <-- ALTERADO para um range maior
    columns=['NOMRES']
)
end_time = time.time()
print(f"Busca por todos os BDRs em Maio de 2023 concluída em {end_time - start_time:.2f} segundos.")
print(f"Encontrados {df_bdrs['CODNEG'].nunique()} BDRs únicos negociados em Maio/2023.")
display(df_bdrs.head())
assert not df_bdrs.empty, "Teste 3.1 Falhou: Busca por asset_class='bdr' em um mês inteiro não retornou dados."
print("[OK] Teste 3.1 Concluído com sucesso.\n" + "-"*80)


print("\n--- TESTE 3.2 (EXTREMO): COMBINAÇÃO DE FILTROS MÚLTIPLOS E COMPLEXOS ---")
# Objetivo: Encontrar opções da Vale usando a abordagem explícita de 'ticker_root'.

# O usuário primeiro descobre o radical (esta etapa pode ser mental ou programática)
ticker_radical_vale = 'VALE' 
print(f"Buscando opções com o radical explícito: '{ticker_radical_vale}'")

df_opcoes_vale = analyzer.get_quotes(
    ticker_root=ticker_radical_vale, # <-- MUDANÇA PRINCIPAL
    asset_class='options',
    vencimento_min='2023-07-01',
    vencimento_max='2023-12-31',
    columns=['PREULT', 'VOLTOT', 'DATVEN', 'PREEXE']
)
print(f"Encontradas {df_opcoes_vale['CODNEG'].nunique()} opções da Vale com vencimento no 2S2023.")
display(df_opcoes_vale.sort_values('VOLTOT', ascending=False).head(10))
assert not df_opcoes_vale.empty, "Teste 3.2 Falhou: Combinação complexa de filtros não retornou dados."
print("[OK] Teste 3.2 Concluído com sucesso.\n" + "-"*80)


print("\n--- TESTE 3.3 (EXTREMO): CASO DE FALHA GRACIOSA ---")
# Objetivo: Garantir que o módulo não quebre ao receber uma query sem resultados.
print("Buscando por uma entidade que não existe...")
df_inexistente = analyzer.get_quotes(entity='Empresa Inexistente XYZ')
print(f"Resultado da busca por entidade inexistente (esperado um DataFrame vazio):")
display(df_inexistente)
assert df_inexistente.empty, "Teste 3.3 Falhou: O módulo deveria retornar um DataFrame vazio."

print("\nBuscando por um ticker sem dados no período...")
df_sem_dados = analyzer.get_quotes(tickers='PETR4', start_date='1980-01-01', end_date='1980-01-31')
print(f"Resultado da busca por dados inexistentes (esperado um DataFrame vazio):")
display(df_sem_dados)
assert df_sem_dados.empty, "Teste 3.3 Falhou: O módulo deveria retornar um DataFrame vazio para períodos sem dados."
print("[OK] Teste 3.3 Concluído com sucesso.\n" + "-"*80)

print("\n\n" + "="*25 + " TODOS OS TESTES FORAM CONCLUÍDOS " + "="*25)